In [7]:
import os
import numpy as np
from matplotlib import pyplot as plt
import tensorflow as tf
import cv2
from tensorflow.keras import layers, models, optimizers, regularizers

In [5]:
# Set up directories
DATASET_PATH = "dataset"
PERSON_NAME = input("Enter person's name: ")  
SAVE_PATH = os.path.join(DATASET_PATH, PERSON_NAME)
os.makedirs(SAVE_PATH, exist_ok=True)

cap = cv2.VideoCapture(0)
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

print("Capturing images... Press 'q' to stop.")

count = 0
while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, 1.3, 5)

    for (x, y, w, h) in faces:
        face = frame[y:y + h, x:x + w]
        face_resized = cv2.resize(face, (224,224))  # Resize to match model input size

        # Save image
        filename = os.path.join(SAVE_PATH, f"{count}.jpg")
        cv2.imwrite(filename, face_resized)
        count += 1

        # Draw rectangle
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)

    cv2.imshow("Face Capture", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

print(f"Captured {count} images for {PERSON_NAME}.")
    

Capturing images... Press 'q' to stop.
Captured 530 images for vishwajith.


In [9]:
datagen = 'C:/Users/vishw/Desktop/face learn/dataset'

train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    datagen,
    label_mode="int",
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(224,224),
    batch_size=32)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    datagen,
    label_mode="int",
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(224,224),
    batch_size=32)

class_names = train_dataset.class_names
print("Class names:", class_names)

# ✅ Data Augmentation
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.2),
    layers.RandomBrightness(0.2),
    layers.RandomContrast(0.2)
])

# ✅ Normalization Layer
normalization_layer = tf.keras.layers.Rescaling(1./255)

# ✅ Apply Data Augmentation + Normalization Together
train_dataset = train_dataset.map(
    lambda x, y: (normalization_layer(data_augmentation(x, training=True)), y)
)

validation_dataset = validation_dataset.map(
    lambda x, y: (normalization_layer(x), y)
)

# ✅ Prefetch for Performance
AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)

# ✅ Model Architecture
model = models.Sequential([
    layers.Conv2D(32, (3,3), activation='relu', kernel_regularizer=regularizers.l2(0.01), input_shape=(224,224,3)),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64, (3,3), activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(128, (3,3), activation='relu', kernel_regularizer=regularizers.l2(0.01)),
    layers.BatchNormalization(),
    layers.MaxPooling2D(2,2),

    layers.GlobalAveragePooling2D(),

    layers.Dense(128, activation='relu'),
    layers.BatchNormalization(),
    layers.Dropout(0.5),

    layers.Dense(len(class_names), activation='softmax')  
])

# ✅ Compile with Lower Learning Rate
model.compile(optimizer=optimizers.Adam(learning_rate=0.0001), 
              loss='sparse_categorical_crossentropy', 
              metrics=['accuracy'])

# ✅ Train with Early Stopping (Increased Patience)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.fit(train_dataset, validation_data=validation_dataset, epochs=20, callbacks=[early_stopping])

# ✅ Save Model
model.save("face_recognition_model_fixed.h5")

print("Model training complete and saved.")


Found 1339 files belonging to 3 classes.
Using 1072 files for training.
Found 1339 files belonging to 3 classes.
Using 267 files for validation.
Class names: ['kishan', 'nayab', 'vishwajith']
Epoch 1/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 27s 724ms/step - accuracy: 0.7254 - loss: 2.0036 - val_accuracy: 0.3708 - val_loss: 2.4159
Epoch 2/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 24s 702ms/step - accuracy: 0.9583 - loss: 1.4724 - val_accuracy: 0.3708 - val_loss: 2.4462
Epoch 3/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 24s 699ms/step - accuracy: 0.9591 - loss: 1.4260 - val_accuracy: 0.3708 - val_loss: 2.5272
Epoch 4/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 25s 739ms/step - accuracy: 0.9818 - loss: 1.3315 - val_accuracy: 0.3708 - val_loss: 2.6903
Epoch 5/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 25s 725ms/step - accuracy: 0.9828 - loss: 1.2924 - val_accuracy: 0.3708 - val_loss: 2.9595
Epoch 6/20
34/34 ━━━━━━━━━━━━━━━━━━━━ 25s 727ms/step - accuracy: 0.9893 - loss: 1.2606 - val_accuracy: 0.3708 - val_loss: 3.3204


Model training complete and saved.


In [17]:
datagen = 'C:/Users/vishw/Desktop/face learn/dataset'


train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    datagen,
    label_mode="int",  # Labels as integers (0 or 1)
    validation_split=0.2,
    subset="training",
    seed=123,
    image_size=(224,224),  # Resize images for faster processing
    batch_size=32)
validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    datagen,
    label_mode="int",  # Labels as integers (0 or 1)
    validation_split=0.2,
    subset="validation",
    seed=123,
    image_size=(224,224),  # Resize images for faster processing
    batch_size=32)

class_names = train_dataset.class_names
print("Class names:", class_names)

# Data Augmentation (only for training)
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.2),
    layers.RandomBrightness(0.2),
    layers.RandomContrast(0.2)
])


normalization_layer = tf.keras.layers.Rescaling(1./255)


train_dataset = train_dataset.map(
    lambda x, y: (data_augmentation(x, training=True), y)  
)
train_dataset = train_dataset.map(
    lambda x, y: (normalization_layer(x), y) 
)

validation_dataset = validation_dataset.map(
    lambda x, y: (normalization_layer(x), y)
)

AUTOTUNE = tf.data.AUTOTUNE
train_dataset = train_dataset.prefetch(buffer_size=AUTOTUNE)
validation_dataset = validation_dataset.prefetch(buffer_size=AUTOTUNE)

# Create CNN model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224,224, 3)),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Conv2D(256, (3, 3), activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.MaxPooling2D(2, 2),

    tf.keras.layers.Flatten(),

    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.7),

    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.5),

    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.BatchNormalization(),
    tf.keras.layers.Dropout(0.3),

    tf.keras.layers.Dense(len(class_names), activation='softmax')  
  
])

# Compile the model
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

model.fit(train_dataset, validation_data=validation_dataset, epochs=20, callbacks=[early_stopping])

# Save the trained model
model.save("face_recognition_model.h5")

print("Model training complete and saved.")


Found 1339 files belonging to 3 classes.
Using 1072 files for training.
Found 1339 files belonging to 3 classes.
Using 267 files for validation.
Class names: ['kishan', 'nayab', 'vishwajith']
Epoch 1/20
 4/34 ━━━━━━━━━━━━━━━━━━━━ 24s 801ms/step - accuracy: 0.3542 - loss: 1.4777

KeyboardInterrupt: 

In [15]:
model = tf.keras.models.load_model("face_recognition_model.h5")

# Load class labels
DATA_DIR = "dataset"
class_names = sorted(os.listdir(DATA_DIR))  # Ensure classes match dataset folder names

# Load Haar cascade
face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + "haarcascade_frontalface_default.xml")

# Open webcam
cap = cv2.VideoCapture(0)

while True:
    ret, frame = cap.read()
    if not ret:
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5, minSize=(30, 30))

    for (x, y, w, h) in faces:
        face = frame[y:y + h, x:x + w]
        face = cv2.resize(face, (224,224))  # Resize to model input size
        face = face / 255.0  # Normalize
        face = np.expand_dims(face, axis=0)  # Add batch dimension

        # Predict class
        predictions = model.predict(face)
        class_id = np.argmax(predictions)
        class_label = class_names[class_id]

        # Draw rectangle and label
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.putText(frame, class_label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.8, (0, 255, 0), 2)

    # Display frame
    cv2.imshow("Live Facial Recognition", frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 141ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 34ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 32ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 33ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step
1/1 ━━━━━━━